In [ ]:
import spacy
import pandas as pd
import numpy as np

In [ ]:
!python -m spacy download it_core_news_lg
nlp = spacy.load("it_core_news_lg") #carico il modello per l'italiano

Calcolo della similarity tra le annotazioni manuali e i risultati dei modelli

In [ ]:
file = pd.read_excel(r'/content/drive/MyDrive/IOcompletion_tesi_agnese_daffara/Dataset/IMPLICIT_results.xlsx')

In [ ]:
#importo i risultati dei modelli e le annotazioni manuali
outputs = [file['BERT-multi'], file['BERT-italian-xxl'], file['UmBERTo'], file['BERTino'], file['ELECTRA-italian']]
predicted1 = file['obj_1']
predicted2 = file['obj_2']

#definisco la funzione per calcolare la similarity
def calculate_similarity(word1, word2):
  word1_token = nlp(word1)
  word2_token = nlp(word2)
  if word2 == "unknown" or word1 == "unknown":
      return 0
  if word1_token.has_vector and word2_token.has_vector:
      similarity = word1_token.similarity(word2_token)
      return similarity
  else:
      return 0

#itero sui risultati di ogni modello per ottenere la similarity con il primo e il secondo annotatore e la similarity media
print("\ncalcolo della similarity\n")
count = 0
for output in outputs:
  count = count+1
  print("modello",count)

  similarity_scores = []
  for pred1, pred2, out in zip(predicted1, predicted2, output):
      similarity1 = calculate_similarity(pred1, out)
      similarity2 = calculate_similarity(pred2, out)
      mean_similarity = np.mean([similarity1, similarity2])
      similarity_scores.append(round(mean_similarity, 2))
  print(similarity_scores)

Lemmatizzazione dei risultati e calcolo della similarity

In [ ]:
def lemmatize_text(text):
    doc = nlp(text)
    lemmatized_text = " ".join([token.lemma_ for token in doc])
    return lemmatized_text

#lemmatizzo le annotazioni manuali
#annotatore 1
predicted_lemmatized1 = []
for i in predicted1:
    predicted_lemmatized1.append(lemmatize_text(i))
#print('primo annotatore =', predicted_lemmatized1)
predicted_lemmatized2 = []
#annotatore2
for i in predicted2:
    predicted_lemmatized2.append(lemmatize_text(i))
#print('secondo annotatore =', predicted_lemmatized2)

print("\nrisultati dei modelli lemmatizzati e similarity\n")
#itero su ogni modello
count = 0
for output in outputs:
  count = count+1
  print("modello",count)

  #lemmatizzo i risultati del modello
  lemmatized = []
  for i in output:
    lemmatized.append(lemmatize_text(i))
  #print("output lemmatizzato =", lemmatized)
  #calcolo la media delle similarity
  similarity_scores = []
  for pred1, pred2, out in zip(predicted_lemmatized1, predicted_lemmatized2, lemmatized):
      similarity1 = calculate_similarity(pred1, out)
      similarity2 = calculate_similarity(pred2, out)
      mean_similarity = np.mean([similarity1, similarity2])
      similarity_scores.append(round(mean_similarity, 2))
  print(similarity_scores)